In [192]:
import pulp
import numpy as np
import matplotlib_inline as plt

In [193]:
prob = pulp.LpProblem("Minimize_Cost", pulp.LpMinimize)

In [194]:
import numpy as np

def generate_random_directed_graph(num_nodes, num_edges, cost_range=(1, 10)):
    if num_edges > (num_nodes - 1) * (num_nodes - 2) // 2 + (num_nodes - 1):
        raise ValueError("Number of edges cannot exceed (num_nodes - 1) * (num_nodes - 2) // 2 + (num_nodes - 1) in a directed graph without mutual connections including node 'r'.")

    # Generate a list of node labels (e.g., 'r', 'a', 'b', 'c', ...)
    nodes = [chr(i) for i in range(97, 96 + num_nodes)]+['r']

    # Generate all possible directed edges without mutual connections, excluding self-loops
    all_possible_edges = [(nodes[i], nodes[j]) for i in range(num_nodes) for j in range(num_nodes) if i != j]
    
    # Remove edges that have mutual connections (both (a, b) and (b, a))
    unique_edges = []
    for edge in all_possible_edges:
        if (edge[1], edge[0]) not in unique_edges:
            unique_edges.append(edge)
    
    # Randomly select a subset of unique edges
    np.random.shuffle(unique_edges)
    selected_edges = unique_edges[:num_edges]

    # Generate a random cost for each edge within the specified range
    edges_with_costs = {edge: np.random.randint(cost_range[0], cost_range[1] + 1) for edge in selected_edges}

    return nodes, edges_with_costs

# Use the function to generate a random directed graph with a controlled number of edges
num_nodes = 5  # For example, 5 nodes including 'r'
num_edges = 9  # Number of edges you want in the graph
nodes, edges_with_costs = generate_random_directed_graph(num_nodes, num_edges)

print("nodes:", nodes)
print("edges_with_costs:", edges_with_costs)

nodes: ['a', 'b', 'c', 'd', 'r']
edges_with_costs: {('c', 'r'): 10, ('c', 'd'): 1, ('b', 'c'): 10, ('a', 'c'): 3, ('a', 'r'): 8, ('a', 'b'): 6, ('b', 'd'): 1, ('a', 'd'): 2, ('b', 'r'): 4}


In [195]:

#nodes = ['a','b','c','d','r']
#edges_with_costs ={
    #('a','b'):3,('c','a'):1,('r','a'):9,('b','c'):8,
    #('b','r'):4,('d','b'):6,('r','c'):2,
    #('c','d'):7,('d','r'):5
#}


In [196]:
import pulp
x = pulp.LpVariable.dicts('x',(nodes,nodes),0,1,cat=pulp.LpInteger)
x

{'a': {'a': x_a_a, 'b': x_a_b, 'c': x_a_c, 'd': x_a_d, 'r': x_a_r},
 'b': {'a': x_b_a, 'b': x_b_b, 'c': x_b_c, 'd': x_b_d, 'r': x_b_r},
 'c': {'a': x_c_a, 'b': x_c_b, 'c': x_c_c, 'd': x_c_d, 'r': x_c_r},
 'd': {'a': x_d_a, 'b': x_d_b, 'c': x_d_c, 'd': x_d_d, 'r': x_d_r},
 'r': {'a': x_r_a, 'b': x_r_b, 'c': x_r_c, 'd': x_r_d, 'r': x_r_r}}

In [197]:
prob +=pulp.lpSum(edges_with_costs[u,v] * x[u][v] for u, v in edges_with_costs)
prob

Minimize_Cost:
MINIMIZE
6*x_a_b + 3*x_a_c + 2*x_a_d + 8*x_a_r + 10*x_b_c + 1*x_b_d + 4*x_b_r + 1*x_c_d + 10*x_c_r + 0
VARIABLES
0 <= x_a_b <= 1 Integer
0 <= x_a_c <= 1 Integer
0 <= x_a_d <= 1 Integer
0 <= x_a_r <= 1 Integer
0 <= x_b_c <= 1 Integer
0 <= x_b_d <= 1 Integer
0 <= x_b_r <= 1 Integer
0 <= x_c_d <= 1 Integer
0 <= x_c_r <= 1 Integer

In [198]:
for v in nodes:
    if v != 'r':
        prob +=pulp.lpSum(x[u][v] for u in nodes if (u,v) in edges_with_costs) == 1
        #prob +=pulp.lpSum(x[u,v] for u in nodes if (v,u) in edges_with_costs) == 1
prob 

Minimize_Cost:
MINIMIZE
6*x_a_b + 3*x_a_c + 2*x_a_d + 8*x_a_r + 10*x_b_c + 1*x_b_d + 4*x_b_r + 1*x_c_d + 10*x_c_r + 0
SUBJECT TO
_C1:0 = 1

_C2: x_a_b = 1

_C3: x_a_c + x_b_c = 1

_C4: x_a_d + x_b_d + x_c_d = 1

VARIABLES
0 <= x_a_b <= 1 Integer
0 <= x_a_c <= 1 Integer
0 <= x_a_d <= 1 Integer
0 <= x_a_r <= 1 Integer
0 <= x_b_c <= 1 Integer
0 <= x_b_d <= 1 Integer
0 <= x_b_r <= 1 Integer
0 <= x_c_d <= 1 Integer
0 <= x_c_r <= 1 Integer

In [203]:
prob += pulp.lpSum(x[u]['r'] for u in nodes if (u,'r') in edges_with_costs) == 0
prob += pulp.lpSum(x['r'][v] for v in nodes if ('r',v) in edges_with_costs)>= 1
prob

Minimize_Cost:
MINIMIZE
6*x_a_b + 3*x_a_c + 2*x_a_d + 8*x_a_r + 10*x_b_c + 1*x_b_d + 4*x_b_r + 1*x_c_d + 10*x_c_r + 0
SUBJECT TO
_C1:0 = 1

_C2: x_a_b = 1

_C3: x_a_c + x_b_c = 1

_C4: x_a_d + x_b_d + x_c_d = 1

_C5: x_a_r + x_b_r + x_c_r = 0

_C6:0 >= 1

_C7: x_a_r + x_b_r + x_c_r = 0

_C8:0 >= 1

VARIABLES
0 <= x_a_b <= 1 Integer
0 <= x_a_c <= 1 Integer
0 <= x_a_d <= 1 Integer
0 <= x_a_r <= 1 Integer
0 <= x_b_c <= 1 Integer
0 <= x_b_d <= 1 Integer
0 <= x_b_r <= 1 Integer
0 <= x_c_d <= 1 Integer
0 <= x_c_r <= 1 Integer

In [200]:
from pulp import *
n = len(nodes)
subsets = [tuple(c) for c in allcombinations(nodes,n-1)]
subset = []
for s in subsets:
    if 'r' in s:
        subset.append(s)
subset

[('r',),
 ('a', 'r'),
 ('b', 'r'),
 ('c', 'r'),
 ('d', 'r'),
 ('a', 'b', 'r'),
 ('a', 'c', 'r'),
 ('a', 'd', 'r'),
 ('b', 'c', 'r'),
 ('b', 'd', 'r'),
 ('c', 'd', 'r'),
 ('a', 'b', 'c', 'r'),
 ('a', 'b', 'd', 'r'),
 ('a', 'c', 'd', 'r'),
 ('b', 'c', 'd', 'r')]

In [201]:
for s in subset:
    sc = list(set(nodes)-set(s)) # complement of subset s
    summation = 0.0
    for u in s:
        for v in sc:
            if (u, v) in edges_with_costs: # Check if the (u, v) tuple is a valid key in edges_with_costs
                summation += x[u][v]
    prob += summation >= 1  

TypeError: A False object cannot be passed as a constraint

In [ ]:
prob.solve()
print ("Status:", LpStatus[prob.status])

In [202]:
print ("Optimal Solution")
for i in nodes:
	for j in nodes:
		if x[i][j].value() == 1:
			print ("(%s,%s)"%(i,j))  


Optimal Solution
